In [ ]:
import subprocess
import neutralb1.utils as utils

WORKSPACE_DIR = utils.get_workspace_dir()

git_hash = subprocess.check_output(['git', 'rev-parse', 'HEAD'], cwd=WORKSPACE_DIR).decode('utf-8').strip()
print(git_hash)

# Input-Output Study
We've now added a spin-3 $F$-wave to our waveset, which was not generated in the signal MC. This way we can test for leakages to the F-wave. Other than this extra wave, the setup is similar to the [standard_results](../spin-1/standard_results.ipynb). 

The details of the generated signal MC and waveset are:
* Generated with $b_1(1235)$ and $\rho(1450)$ Breit-Wigners fixed to their PDG values
  * an isotropic background is included, but is so small it's negligible
* The fit results here are from mass-independent fits (no Breit-Wigners) to the signal generated by these BW's.
* Fit with a waveset $J^P\ell = \{1^+S, 1^+D, 1^-P, 3^-F\}
* No `OmegaDalitz` amplitudes for changing the dalitz distribution of the $\omega$ and its corresponding $\lambda$ distribution
* No $D/S$ ratio that would be typically associated with the $b_1$
* Only in the PARA 0 orientation

Each bin of mass has 500 randomized fits and 100 bootstrap fits 

In [ ]:
# load common libraries
import pandas as pd
import pickle as pkl
import pathlib
import os, sys
import numpy as np
import matplotlib.pyplot as plt
from typing import Dict

# load neutralb1 libraries
import neutralb1.utils as utils
from neutralb1.analysis.result import ResultManager
import neutralb1.analysis.statistics as stats

utils.load_environment()

# load in useful directories as constants
CWD = pathlib.Path.cwd()
STUDY_DIR = f"{WORKSPACE_DIR}/studies/io-tests/spin-3/"

# set env variables for shell cells
os.environ["WORKSPACE_DIR"] = WORKSPACE_DIR
os.environ['STUDY_DIR'] = STUDY_DIR

In [ ]:
%%bash
# print out yaml file used to submit the fits
cat $STUDY_DIR/submission.YAML

In [ ]:
# load in preprocessed results
with open(f"{STUDY_DIR}/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data = pkl.load(f)
results = ResultManager(**data)

In [ ]:
results.summary()

## Analysis

### Standard Plots

In [ ]:
results.plot.intensity.jp()
plt.show()

In [ ]:
results.plot.intensity.waves()
plt.show()

In [ ]:
results.plot.intensity.waves(fractional=True)
plt.show()

In [ ]:
results.plot.intensity.waves(reflectivity="negative")
plt.savefig(f"{STUDY_DIR}/plots/waves_negative.pdf")

In [ ]:
results.plot.diagnostic.matrix()
plt.show()

In [ ]:
sig_moments = list(results.get_significant_moments(threshold=0.02))
results.plot.intensity.moments(moments=sig_moments)
plt.savefig(f"{STUDY_DIR}/plots/significant_moments.pdf")

### Naturalities

First we'll just do the total naturality contribution

In [ ]:
results.plot.intensity.plot(
    ["p", "m"], 
    fractional=True, 
    col_kwargs={"p": {"color":"red", "label":"Natural"}, "m": {"color":"blue", "label":"Unnatural"}})
plt.savefig(f"{STUDY_DIR}/plots/p_m_fractional.pdf")

### b1 and rho(1450) interference

In [ ]:
colors = plt.colormaps["Dark2"].colors # type: ignore # match colors to JP plot
results.plot.phase.mass_phase(
    "p1p0S", "p1mpP", 
    amp1_kwargs={"color": colors[2]},
    amp2_kwargs={"color": colors[3]},
    )
plt.savefig(f"{STUDY_DIR}/plots/interference.pdf")

Lets make a couple joyplots to see how the bootstrap distributions are shaped

In [ ]:
results.plot.bootstrap.joyplot(
    ["p1p0S", "p1mpP"],
)
plt.show()

In [ ]:
results.plot.bootstrap.joyplot(
    [results.phase_difference_dict[("p1p0S", "p1mpP")]]    
)
plt.show()

## Comparison to Standard Results
Since this is fit to the same dataset, we can directly compare the values we obtain in each fit

In [ ]:
# first load the standard data
with open(f"{STUDY_DIR}/../spin-1/t_0.1-1.0/preprocessed_results_acceptance_corrected.pkl", "rb") as f:
    data_standard = pkl.load(f)
results_standard = ResultManager(**data_standard)

### Likelihood Ratio

In [ ]:
# [2 (real, imag) params per amp] * [2 (reflectivities)] * [3 (spin projections)] * [3 (angular momenta: S, P, D)] - [2 reference phases]
standard_free_params = 2 * 2 * 3 * 3 - 2

In [ ]:
# add [2 (real, imag) params per amp] * [2 (reflectivities)] * [5 (spin projections)] * [1 angular momenta: F]
spin3_free_params = standard_free_params + 2 * 2 * 5 * 1

In [ ]:
results.plot.diagnostic.lrt_pvalues(
    results_standard,
    spin3_free_params,
    standard_free_params,
)
plt.savefig(f"{STUDY_DIR}/plots/lrt_pvalues.pdf", bbox_inches='tight')

### Moments

In [ ]:
assert results.proj_moments_df is not None
assert results_standard.proj_moments_df is not None
moments = [c for c in results_standard.proj_moments_df.columns if c.startswith("H")]
moments.remove("H0_0000")
n_moments = len(moments)
fig, axs = plt.subplots(
    nrows=int(np.ceil(n_moments / 3)),
    ncols=min(3, n_moments),
    figsize=(15, 5 * np.ceil(n_moments / 3)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)

for ax, col in zip(axs.flatten(), moments):
    results.plot.intensity.plot(
        [col], ax=ax,
        col_kwargs={col: {"markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":"spin-3"}}
    )
    results_standard.plot.intensity.plot(
        [col], ax=ax,
        col_kwargs={col: {"color":"green", "marker":"o", "label":"spin-1"}}
    )    
    ax.set_title(utils.convert_moment_name(col))    
    ax.legend()

#### Significant Moments
Lets just do the significant ones now to present

In [ ]:
sig_moments = results.get_significant_moments(threshold=0.02).union(results_standard.get_significant_moments(threshold=0.02))
sig_moments.remove("H0_0000") # always 1 for fit fractions

n_moments = len(sig_moments)
fig, axs = plt.subplots(
    nrows=int(np.ceil(n_moments / 4)),
    ncols=min(4, n_moments),
    figsize=(15, 5 * np.ceil(n_moments / 4)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)

for i, (ax, col) in enumerate(zip(axs.flatten(), sig_moments)):
    results.plot.intensity.plot(
        [col], ax=ax,
        col_kwargs={col: {"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":"spin-3", "alpha": 0.7}}
    )
    results_standard.plot.intensity.plot(
        [col], ax=ax,
        col_kwargs={col: {"color":"green", "marker":"o", "label":"spin-1", "alpha": 0.7}}
    )    
    ax.set_title(utils.convert_moment_name(col))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

plt.savefig(f"{STUDY_DIR}/plots/significant_moments_comparison.pdf")

### All Amplitudes

In [ ]:
amps = results_standard.coherent_sums["eJPmL"]
n_amps = len(amps)

fig, axs = plt.subplots(
    nrows=int(np.ceil(n_amps / 4)),
    ncols=min(4, n_amps),
    figsize=(15, 5 * np.ceil(n_amps / 4)),
    sharex=True,
    squeeze=False,
    layout="constrained"
)

for i, (ax, amp) in enumerate(zip(axs.flatten(), amps)):
    results.plot.intensity.plot(
        [amp], ax=ax,
        col_kwargs={amp: {"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":"spin-3", "alpha": 0.7}}
    )
    results_standard.plot.intensity.plot(
        [amp], ax=ax,
        col_kwargs={amp: {"color":"green", "marker":"o", "label":"spin-1", "alpha": 0.7}}
    )        
    ax.set_title(utils.convert_amp_name(amp))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

for ax in axs.flatten()[n_amps:]:
    ax.set_visible(False)

### Phases
Since the unnatural phases are heavily dominated by error and there are tons of natural phases we could choose, we'll simply choose phases between the most significant amplitudes (using a 2% threshold)

In [ ]:
phases = results_standard.get_significant_phases(threshold=0.02)
n_phases = len(phases)

fig, axs = plt.subplots(
    nrows=int(np.ceil(n_phases / 4)),
    ncols=min(4, n_phases),
    figsize=(15, 5 * np.ceil(n_phases / 4)),
    sharex=True,    
    squeeze=False,
    layout="constrained"
)
for i, (ax, phase) in enumerate(zip(axs.flatten(), phases)):
    amp1, amp2 = phase.split("_")
    results.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"black", "markeredgecolor":"black", "marker":"o", "markerfacecolor":"white", "label":"spin-3", "alpha": 0.7},
        extend_range=False
    )
    results_standard.plot.phase.phase(
        amp1, amp2, ax=ax,
        phase_kwargs={"color":"green", "marker":"o", "label":"spin-1", "alpha": 0.7},        
        extend_range=False
    )    
    ax.set_title(utils.convert_amp_name(phase))
    ax.legend()
    if i != 0:
        ax.get_legend().remove()
    if i % 4 != 0:
        ax.set_ylabel("")

for ax in axs.flatten()[n_phases:]:
    ax.set_visible(False)

## Cleanup
zip all the plots together

In [ ]:
%%bash
cd $STUDY_DIR/plots
zip -u plots.zip *.pdf